<a href="https://colab.research.google.com/github/MohithAnkem/Milestone3/blob/main/CS297_Landslide_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install rasterio gdal


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.0 MB/s eta 0:00:00


In [3]:
!pip install numpy pandas opencv-python tensorflow matplotlib rasterio gdal


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
import os
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
dataset_path = '/content/drive/MyDrive/Landslide_Dataset/'

os.listdir(dataset_path)


['Wenchuan.zip',
 'Tiburon Peninsula（planet）.zip',
 'Lombok.zip',
 'Jiuzhai valley (UAV-0.5m).zip',
 'Longxi River(SAT).zip',
 'Moxi town（UAV-1m）.zip',
 'Hokkaido Iburi-Tobu.zip',
 'Longxi River（UAV）.zip',
 'palu.zip',
 'Moxitaidi (SAT).zip',
 'Tiburon Peninsula（Sentinel）.zip',
 'Moxitaidi (UAV-1m).zip',
 'Moxitaidi (UAV-0.6m).zip',
 'Hokkaido Iburi-Tobu',
 'Jiuzhai valley (UAV-0.5m)',
 'Wenchuan',
 'Tiburon Peninsula（planet）',
 'Lombok',
 'palu',
 'split']

extracting individual folder

In [ ]:
import zipfile

# Unzip a specific region (e.g., Region1)
with zipfile.ZipFile(dataset_path + 'Region1.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset/Region1')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Landslide_Dataset/Region1.zip'

for CPU

In [ ]:
import zipfile

# Unzip all region zip files
for zip_file in os.listdir(dataset_path):
    if zip_file.endswith('.zip'):
        with zipfile.ZipFile(dataset_path + zip_file, 'r') as zip_ref:
            zip_ref.extractall('/content/drive/MyDrive/Landslide_Dataset/' + zip_file[:-4])

for GPU

In [6]:
import zipfile

# Unzip all region zip files
for zip_file in os.listdir(dataset_path):
    if zip_file.endswith('.zip'):
        with zipfile.ZipFile(dataset_path + zip_file, 'r') as zip_ref:
            zip_ref.extractall('/content/Land_Dataset/' + zip_file[:-4])

In [7]:
# List the contents of the extracted Region1 folder
os.listdir('/content/Land_Dataset/Wenchuan/img/')

['wenchuan115.tif',
 'wenchuan140.tif',
 'wenchuan074.tif',
 'wenchuan037.tif',
 'wenchuan130.tif',
 'wenchuan131.tif',
 'wenchuan069.tif',
 'wenchuan053.tif',
 'wenchuan008.tif',
 'wenchuan091.tif',
 'wenchuan022.tif',
 'wenchuan070.tif',
 'wenchuan007.tif',
 'wenchuan165.tif',
 'wenchuan043.tif',
 'wenchuan019.tif',
 'wenchuan044.tif',
 'wenchuan009.tif',
 'wenchuan030.tif',
 'wenchuan067.tif',
 'wenchuan137.tif',
 'wenchuan041.tif',
 'wenchuan025.tif',
 'wenchuan111.tif',
 'wenchuan047.tif',
 'wenchuan045.tif',
 'wenchuan082.tif',
 'wenchuan004.tif',
 'wenchuan172.tif',
 'wenchuan077.tif',
 'wenchuan124.tif',
 'wenchuan177.tif',
 'wenchuan101.tif',
 'wenchuan135.tif',
 'wenchuan122.tif',
 'wenchuan085.tif',
 'wenchuan049.tif',
 'wenchuan052.tif',
 'wenchuan170.tif',
 'wenchuan138.tif',
 'wenchuan104.tif',
 'wenchuan031.tif',
 'wenchuan076.tif',
 'wenchuan116.tif',
 'wenchuan159.tif',
 'wenchuan112.tif',
 'wenchuan003.tif',
 'wenchuan079.tif',
 'wenchuan110.tif',
 'wenchuan136.tif',


In [ ]:
# List the contents of the extracted Region1 folder
os.listdir('/content/drive/MyDrive/Landslide_Dataset/Lombok/img/')

Cropping the images in the daaset into 512x512 pixel size as a part of preprocessing

for CPU

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None

crop_height = 512
crop_width = 512
stride = 256
overlap = int(crop_width - stride)
img_predict_dir = '/content/drive/MyDrive/Landslide_Dataset/Lombok/img/'
img_result_dir = '/content/drive/MyDrive/Landslide_Dataset/Lombok/cropped_img/'

if not os.path.exists(img_result_dir):
    os.makedirs(img_result_dir)

num = 1
print(img_result_dir)


def fuse(output_image, row, column, overlap, counter, temp1, temp2, res_row, res_column):
    if counter % column == 0:
        output_image = output_image[:, -(res_column + overlap):]
    if counter > (row - 1) * column:
        output_image = output_image[-(res_row + overlap):, :]

    if counter % column == 1:
        temp1 = output_image
    else:
        temp1_1 = temp1[:, 0:-overlap, :]
        temp1_2 = temp1[:, -overlap:, :]
        temp1_3 = output_image[:, 0:overlap, :]
        temp1_4 = output_image[:, overlap:, :]
        temp1_fuse = 0.5 * temp1_2 + 0.5 * temp1_3
        temp1 = np.concatenate((temp1_1, temp1_fuse, temp1_4), axis=1)
        if int(counter % column) == 0:
            if counter == column:
                temp2 = temp1
            else:
                temp2_1 = temp2[0:-overlap, :, :]
                temp2_2 = temp2[-overlap:, :, :]
                temp2_3 = temp1[0:overlap, :, :]
                temp2_4 = temp1[overlap:, :, :]
                temp2_fuse = 0.5 * temp2_2 + 0.5 * temp2_3
                temp2 = np.concatenate((temp2_1, temp2_fuse, temp2_4), axis=0)

    return temp1, temp2


for filename in os.listdir(img_predict_dir):
    img_name = os.path.join(img_predict_dir, filename)
    print(img_name)
    loaded_image = Image.open(img_name)
    img_w, img_h = loaded_image.size
    row = int((img_h - crop_height) / stride + 1)
    column = int((img_w - crop_width) / stride + 1)
    res_row = img_h - (crop_height + stride * (row - 1))
    res_column = img_w - (crop_width + stride * (column - 1))
    if (img_h - crop_height) % stride > 0:
        row += 1
    if (img_w - crop_width) % stride > 0:
        column += 1
    counter = 1

    for i in range(row):
        for j in range(column):
            if i == row - 1:
                H_start = img_h - crop_height
                H_end = img_h
            else:
                H_start = i * stride
                H_end = H_start + crop_height
            if j == column - 1:
                W_start = img_w - crop_width
                W_end = img_w
            else:
                W_start = j * stride
                W_end = W_start + crop_width

            img_chip = loaded_image.crop((W_start, H_start, W_end, H_end))
            img_chip.save(os.path.join(img_result_dir, f'{num:04}.tif'))
            num += 1

for GPU anf for a single region - cropping

In [8]:
import os
import cv2
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None

crop_height = 512
crop_width = 512
stride = 256
overlap = int(crop_width - stride)
img_predict_dir = '/content/Land_Dataset/Wenchuan/img/'
img_result_dir = '/content/Land_Dataset/Wenchuan/cropped_img/'

if not os.path.exists(img_result_dir):
    os.makedirs(img_result_dir)

num = 1
print(img_result_dir)


def fuse(output_image, row, column, overlap, counter, temp1, temp2, res_row, res_column):
    if counter % column == 0:
        output_image = output_image[:, -(res_column + overlap):]
    if counter > (row - 1) * column:
        output_image = output_image[-(res_row + overlap):, :]

    if counter % column == 1:
        temp1 = output_image
    else:
        temp1_1 = temp1[:, 0:-overlap, :]
        temp1_2 = temp1[:, -overlap:, :]
        temp1_3 = output_image[:, 0:overlap, :]
        temp1_4 = output_image[:, overlap:, :]
        temp1_fuse = 0.5 * temp1_2 + 0.5 * temp1_3
        temp1 = np.concatenate((temp1_1, temp1_fuse, temp1_4), axis=1)
        if int(counter % column) == 0:
            if counter == column:
                temp2 = temp1
            else:
                temp2_1 = temp2[0:-overlap, :, :]
                temp2_2 = temp2[-overlap:, :, :]
                temp2_3 = temp1[0:overlap, :, :]
                temp2_4 = temp1[overlap:, :, :]
                temp2_fuse = 0.5 * temp2_2 + 0.5 * temp2_3
                temp2 = np.concatenate((temp2_1, temp2_fuse, temp2_4), axis=0)

    return temp1, temp2


for filename in os.listdir(img_predict_dir):
    img_name = os.path.join(img_predict_dir, filename)
    print(img_name)
    loaded_image = Image.open(img_name)
    img_w, img_h = loaded_image.size
    row = int((img_h - crop_height) / stride + 1)
    column = int((img_w - crop_width) / stride + 1)
    res_row = img_h - (crop_height + stride * (row - 1))
    res_column = img_w - (crop_width + stride * (column - 1))
    if (img_h - crop_height) % stride > 0:
        row += 1
    if (img_w - crop_width) % stride > 0:
        column += 1
    counter = 1

    for i in range(row):
        for j in range(column):
            if i == row - 1:
                H_start = img_h - crop_height
                H_end = img_h
            else:
                H_start = i * stride
                H_end = H_start + crop_height
            if j == column - 1:
                W_start = img_w - crop_width
                W_end = img_w
            else:
                W_start = j * stride
                W_end = W_start + crop_width

            img_chip = loaded_image.crop((W_start, H_start, W_end, H_end))
            img_chip.save(os.path.join(img_result_dir, f'{num:04}.tif'))
            num += 1

/content/Land_Dataset/Wenchuan/cropped_img/
/content/Land_Dataset/Wenchuan/img/wenchuan115.tif
/content/Land_Dataset/Wenchuan/img/wenchuan140.tif
/content/Land_Dataset/Wenchuan/img/wenchuan074.tif
/content/Land_Dataset/Wenchuan/img/wenchuan037.tif
/content/Land_Dataset/Wenchuan/img/wenchuan130.tif
/content/Land_Dataset/Wenchuan/img/wenchuan131.tif
/content/Land_Dataset/Wenchuan/img/wenchuan069.tif
/content/Land_Dataset/Wenchuan/img/wenchuan053.tif
/content/Land_Dataset/Wenchuan/img/wenchuan008.tif
/content/Land_Dataset/Wenchuan/img/wenchuan091.tif
/content/Land_Dataset/Wenchuan/img/wenchuan022.tif
/content/Land_Dataset/Wenchuan/img/wenchuan070.tif
/content/Land_Dataset/Wenchuan/img/wenchuan007.tif
/content/Land_Dataset/Wenchuan/img/wenchuan165.tif
/content/Land_Dataset/Wenchuan/img/wenchuan043.tif
/content/Land_Dataset/Wenchuan/img/wenchuan019.tif
/content/Land_Dataset/Wenchuan/img/wenchuan044.tif
/content/Land_Dataset/Wenchuan/img/wenchuan009.tif
/content/Land_Dataset/Wenchuan/img/wen

In [9]:
os.listdir('/content/Land_Dataset/Wenchuan/cropped_img/')

['0156.tif',
 '0126.tif',
 '0121.tif',
 '0119.tif',
 '0146.tif',
 '0016.tif',
 '0120.tif',
 '0030.tif',
 '0021.tif',
 '0001.tif',
 '0159.tif',
 '0052.tif',
 '0087.tif',
 '0025.tif',
 '0173.tif',
 '0073.tif',
 '0094.tif',
 '0084.tif',
 '0007.tif',
 '0122.tif',
 '0022.tif',
 '0127.tif',
 '0099.tif',
 '0098.tif',
 '0097.tif',
 '0015.tif',
 '0079.tif',
 '0143.tif',
 '0150.tif',
 '0045.tif',
 '0140.tif',
 '0020.tif',
 '0086.tif',
 '0111.tif',
 '0162.tif',
 '0059.tif',
 '0155.tif',
 '0139.tif',
 '0036.tif',
 '0148.tif',
 '0064.tif',
 '0131.tif',
 '0112.tif',
 '0014.tif',
 '0114.tif',
 '0066.tif',
 '0058.tif',
 '0092.tif',
 '0010.tif',
 '0034.tif',
 '0006.tif',
 '0115.tif',
 '0158.tif',
 '0107.tif',
 '0151.tif',
 '0023.tif',
 '0132.tif',
 '0005.tif',
 '0088.tif',
 '0081.tif',
 '0060.tif',
 '0161.tif',
 '0138.tif',
 '0136.tif',
 '0049.tif',
 '0178.tif',
 '0133.tif',
 '0089.tif',
 '0008.tif',
 '0082.tif',
 '0076.tif',
 '0039.tif',
 '0118.tif',
 '0110.tif',
 '0152.tif',
 '0116.tif',
 '0144.tif',

In [ ]:
for filename in os.listdir(img_predict_dir):
    img_name = os.path.join(img_predict_dir, filename)
    print(f"Processing {img_name}...")
    loaded_image = Image.open(img_name)
    img_w, img_h = loaded_image.size
    row = int((img_h - crop_height) / stride + 1)
    column = int((img_w - crop_width) / stride + 1)
    res_row = img_h - (crop_height + stride * (row - 1))
    res_column = img_w - (crop_width + stride * (column - 1))
    if (img_h - crop_height) % stride > 0:
        row += 1
    if (img_w - crop_width) % stride > 0:
        column += 1
    counter = 1

    for i in range(row):
        for j in range(column):
            if i == row - 1:
                H_start = img_h - crop_height
                H_end = img_h
            else:
                H_start = i * stride
                H_end = H_start + crop_height
            if j == column - 1:
                W_start = img_w - crop_width
                W_end = img_w
            else:
                W_start = j * stride
                W_end = W_start + crop_width

            img_chip = loaded_image.crop((W_start, H_start, W_end, H_end))
            print(f"Saving cropped image {num} with size {img_chip.size} at {os.path.join(img_result_dir, f'{num:04}.tif')}")
            img_chip.save(os.path.join(img_result_dir, f'{num:04}.tif'))
            num += 1


Processing /content/Land_Dataset/Lombok/img/Lombok307.tif...
Saving cropped image 437 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0437.tif
Processing /content/Land_Dataset/Lombok/img/Lombok266.tif...
Saving cropped image 438 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0438.tif
Processing /content/Land_Dataset/Lombok/img/Lombok256.tif...
Saving cropped image 439 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0439.tif
Processing /content/Land_Dataset/Lombok/img/Lombok488.tif...
Saving cropped image 440 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0440.tif
Processing /content/Land_Dataset/Lombok/img/Lombok287.tif...
Saving cropped image 441 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0441.tif
Processing /content/Land_Dataset/Lombok/img/Lombok038.tif...
Saving cropped image 442 with size (512, 512) at /content/Land_Dataset/Lombok/cropped_img/0442.tif
Processing /content/Land_Dataset/Lombok/

for CPU - dividing the dataset into train , test and validation

In [ ]:
import os
import random
import shutil
import math

def moveFile(train_img_Dir, train_mask_Dir, train_img_out_Dir, train_mask_out_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir):
    img_pathDir = os.listdir(train_img_Dir)
    random.shuffle(img_pathDir)  # Shuffle the list of image files

    total_files = len(img_pathDir)

    # Set split ratios
    train_rate = 0.7
    val_rate = 0.15
    test_rate = 0.15

    # Calculate the number of files for each set
    train_picknumber = math.floor(total_files * train_rate)
    val_picknumber = math.floor(total_files * val_rate)
    test_picknumber = total_files - train_picknumber - val_picknumber

    # Split files
    train_files = img_pathDir[:train_picknumber]
    val_files = img_pathDir[train_picknumber:train_picknumber + val_picknumber]
    test_files = img_pathDir[train_picknumber + val_picknumber:]

    """# Move training files
    move_files(train_files, train_img_Dir, train_img_out_Dir)
    move_files(train_files, train_mask_Dir, train_mask_out_Dir)

    # Move validation files
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files(val_files, train_mask_Dir, val_mask_Dir)

    # Move test files
    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files(test_files, train_mask_Dir, test_mask_Dir)"""

    # Move training files
    move_files(train_files, train_img_Dir, train_img_out_Dir)
    move_files_with_mask_check(train_files, train_img_Dir, train_mask_Dir, train_mask_out_Dir)

    # Move validation files
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files_with_mask_check(val_files, train_img_Dir, train_mask_Dir, val_mask_Dir)

    # Move test files
    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files_with_mask_check(test_files, train_img_Dir, train_mask_Dir, test_mask_Dir)


def move_files(files, source_dir, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for filename in files:
        name, ext = os.path.splitext(filename)
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, filename)

        # Handle the case where the destination file already exists
        if os.path.exists(destination_file):
            print(f"Warning: Skipping file '{filename}' as it already exists in the destination directory.")
            continue

        shutil.move(source_file, destination_file)

def move_files_with_mask_check(files, img_source_dir, mask_source_dir, mask_destination_dir):
    """
    Moves mask files, checking if they exist before moving and ignoring if not found.
    """
    if not os.path.exists(mask_destination_dir):
        os.makedirs(mask_destination_dir)

    for filename in files:
        # Assuming mask files have the same name as image files
        mask_filename = filename
        mask_source_file = os.path.join(mask_source_dir, mask_filename)
        mask_destination_file = os.path.join(mask_destination_dir, mask_filename)

        if os.path.exists(mask_source_file):
            # Move the mask file if it exists
            if not os.path.exists(mask_destination_file): # Avoid overwriting existing files
                shutil.move(mask_source_file, mask_destination_file)
            else:
                print(f"Warning: Skipping mask file '{mask_filename}' as it already exists in the destination directory.")
        else:
            print(f"Warning: Mask file '{mask_filename}' not found. Skipping.")  # Print a warning and skip


if __name__ == '__main__':
    # Paths to original dataset
    # Path to Lombok cropped images and masks
    train_img_Dir = '/content/drive/MyDrive/Landslide_Dataset/Lombok/cropped_img/'
    train_mask_Dir = '/content/drive/MyDrive/Landslide_Dataset/Lombok/mask/'

# Paths to output directories for the split dataset
    train_img_out_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/train/img/'
    train_mask_out_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/train/mask/'

    test_img_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/test/img/'
    test_mask_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/test/mask/'

    val_img_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/val/img/'
    val_mask_Dir = '/content/drive/MyDrive/Landslide_Dataset/split/Lombok/val/mask/'


    # Call the function to move files
    moveFile(train_img_Dir, train_mask_Dir, train_img_out_Dir, train_mask_out_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir)


for GPU

In [10]:
import os
import random
import shutil
import math

def moveFile(train_img_Dir, train_mask_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir):
    img_pathDir = os.listdir(train_img_Dir)
    random.shuffle(img_pathDir)  # Shuffle the list of image files

    total_files = len(img_pathDir)
    test_rate = 0.7
    val_rate = 0.3

    test_picknumber = math.floor(total_files * test_rate)
    val_picknumber = math.floor(total_files * val_rate)

    test_files = img_pathDir[:test_picknumber]
    val_files = img_pathDir[test_picknumber:test_picknumber + val_picknumber]

    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files(test_files, train_mask_Dir, test_mask_Dir)
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files(val_files, train_mask_Dir, val_mask_Dir)

def move_files(files, source_dir, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    files_lower = [f.lower() for f in files]
    source_files_lower = {f.lower(): f for f in os.listdir(source_dir)}

    for filename in files_lower:
        #name, ext = os.path.splitext(filename)
        original_filename = source_files_lower.get(filename)
        if not original_filename:
            print(f"Warning: '{filename}' not found in source directory.")
            continue
        source_file = os.path.join(source_dir, original_filename)
        destination_file = os.path.join(destination_dir, original_filename)

        # Handle the case where the destination file already exists
        if os.path.exists(destination_file):
            print(f"Warning: Skipping file '{filename}' as it already exists in the destination directory.")
            continue

        shutil.move(source_file, destination_file)

if __name__ == '__main__':
    train_img_Dir = '/content/Land_Dataset/Wenchuan/img/' #The original unpartitioned dataset
    train_mask_Dir = '/content/Land_Dataset/Wenchuan/mask/' #The original unpartitioned dataset
    test_img_Dir = '/content/Land_Dataset/Wenchuan/test/img/'
    test_mask_Dir = '/content/Land_Dataset/Wenchuan/test/mask/'
    val_img_Dir = '/content/Land_Dataset/Wenchuan/val/img/'
    val_mask_Dir = '/content/Land_Dataset/Wenchuan/val/mask/'

    moveFile(train_img_Dir, train_mask_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir)

train,test,val

In [12]:
import os
import random
import shutil
import math

def moveFile(train_img_Dir, train_mask_Dir, train_img_out_Dir, train_mask_out_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir):
    img_pathDir = os.listdir(train_img_Dir)
    random.shuffle(img_pathDir)  # Shuffle the list of image files

    total_files = len(img_pathDir)

    # Set split ratios
    train_rate = 0.7
    val_rate = 0.15
    test_rate = 0.15

    # Calculate the number of files for each set
    train_picknumber = math.floor(total_files * train_rate)
    val_picknumber = math.floor(total_files * val_rate)
    test_picknumber = total_files - train_picknumber - val_picknumber

    # Split files
    train_files = img_pathDir[:train_picknumber]
    val_files = img_pathDir[train_picknumber:train_picknumber + val_picknumber]
    test_files = img_pathDir[train_picknumber + val_picknumber:]

    """# Move training files
    move_files(train_files, train_img_Dir, train_img_out_Dir)
    move_files(train_files, train_mask_Dir, train_mask_out_Dir)

    # Move validation files
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files(val_files, train_mask_Dir, val_mask_Dir)

    # Move test files
    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files(test_files, train_mask_Dir, test_mask_Dir)"""

    # Move training files
    move_files(train_files, train_img_Dir, train_img_out_Dir)
    move_files_with_mask_check(train_files, train_img_Dir, train_mask_Dir, train_mask_out_Dir)

    # Move validation files
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files_with_mask_check(val_files, train_img_Dir, train_mask_Dir, val_mask_Dir)

    # Move test files
    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files_with_mask_check(test_files, train_img_Dir, train_mask_Dir, test_mask_Dir)


def move_files(files, source_dir, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for filename in files:
        name, ext = os.path.splitext(filename)
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, filename)

        # Handle the case where the destination file already exists
        if os.path.exists(destination_file):
            print(f"Warning: Skipping file '{filename}' as it already exists in the destination directory.")
            continue

        shutil.move(source_file, destination_file)

def move_files_with_mask_check(files, img_source_dir, mask_source_dir, mask_destination_dir):
    """
    Moves mask files, checking if they exist before moving and ignoring if not found.
    """
    if not os.path.exists(mask_destination_dir):
        os.makedirs(mask_destination_dir)

    for filename in files:
        # Assuming mask files have the same name as image files
        mask_filename = filename
        mask_source_file = os.path.join(mask_source_dir, mask_filename)
        mask_destination_file = os.path.join(mask_destination_dir, mask_filename)

        if os.path.exists(mask_source_file):
            # Move the mask file if it exists
            if not os.path.exists(mask_destination_file): # Avoid overwriting existing files
                shutil.move(mask_source_file, mask_destination_file)
            else:
                print(f"Warning: Skipping mask file '{mask_filename}' as it already exists in the destination directory.")
        else:
            print(f"Warning: Mask file '{mask_filename}' not found. Skipping.")  # Print a warning and skip


if __name__ == '__main__':
    # Paths to original dataset
    # Path to Lombok cropped images and masks
    train_img_Dir = '/content/Land_Dataset/Wenchuan/cropped_img/'
    train_mask_Dir = '/content/Land_Dataset/Wenchuan/mask/'

# Paths to output directories for the split dataset
    train_img_out_Dir = '/content/Land_Dataset/split/Wenchuan/train/img/'
    train_mask_out_Dir = '/content/Land_Dataset/split/Wenchuan/train/mask/'

    test_img_Dir = '/content/Land_Dataset/split/Wenchuan/test/img/'
    test_mask_Dir = '/content/Land_Dataset/split/Wenchuan/test/mask/'

    val_img_Dir = '/content/Land_Dataset/split/Wenchuan/val/img/'
    val_mask_Dir = '/content/Land_Dataset/split/Wenchuan/val/mask/'


    # Call the function to move files
    moveFile(train_img_Dir, train_mask_Dir, train_img_out_Dir, train_mask_out_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir)


In [ ]:
import os
import random
import shutil
import math

def moveFile(train_img_Dir, train_mask_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir):
    img_pathDir = os.listdir(train_img_Dir)
    random.shuffle(img_pathDir)  # Shuffle the list of image files

    total_files = len(img_pathDir)
    train_rate = 0.7
    val_rate = 0.15
    test_rate = 0.15

    train_picknumber = math.floor(total_files * train_rate)
    val_picknumber = math.floor(total_files * val_rate)
    test_picknumber = total_files - train_picknumber - val_picknumber

    '''test_picknumber = math.floor(total_files * test_rate)
    val_picknumber = math.floor(total_files * val_rate)'''

    test_files = img_pathDir[:test_picknumber]
    val_files = img_pathDir[test_picknumber:test_picknumber + val_picknumber]

    move_files(test_files, train_img_Dir, test_img_Dir)
    move_files(test_files, train_mask_Dir, test_mask_Dir)
    move_files(val_files, train_img_Dir, val_img_Dir)
    move_files(val_files, train_mask_Dir, val_mask_Dir)

def move_files(files, source_dir, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for filename in files:
        name, ext = os.path.splitext(filename)
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, filename)

        # Handle the case where the destination file already exists
        if os.path.exists(destination_file):
            print(f"Warning: Skipping file '{filename}' as it already exists in the destination directory.")
            continue

        shutil.move(source_file, destination_file)

if __name__ == '__main__':
    train_img_Dir = '/content/Land_Dataset/Lombok/cropped_img/' #The original unpartitioned dataset
    train_mask_Dir = '/content/Land_Dataset/Lombok/mask/' #The original unpartitioned dataset
    test_img_Dir = '/content/Land_Dataset/split/Lombok/test/img/'
    test_mask_Dir = '/content/Land_Dataset/split/Lombok/test/mask/'
    val_img_Dir = '/content/Land_Dataset/split/Lombok/val/img/'
    val_mask_Dir = '/content/Land_Dataset/split/Lombok/val/mask/'

    moveFile(train_img_Dir, train_mask_Dir, test_img_Dir, test_mask_Dir, val_img_Dir, val_mask_Dir)

In [11]:
import numpy as np
import os
from PIL import Image
from joblib import Parallel, delayed
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class ImageOperation:

    def __init__(self):
        pass

    @staticmethod
    def openImage(image):
        return Image.open(image, mode="r")

    @staticmethod
    def saveImage(image, path):
        image.save(path)

def label_annotation(img):
    label = np.zeros([img.shape[0], img.shape[1]], dtype=np.uint8)
    black_indices = np.where((img[:, :, 0] == 0) & (img[:, :, 1] == 0) & (img[:, :, 2] == 0))
    white_indices = np.where((img[:, :, 0] == 255) & (img[:, :, 1] == 255) & (img[:, :, 2] == 255))
    label[black_indices[0], black_indices[1]] = 1  # back=1
    label[white_indices[0], white_indices[1]] = 0  # white=0
    return Image.fromarray(np.uint8(label))

def process_color(color_path, label_path, color_name):
    tmp_color_name = os.path.join(color_path, color_name)
    color_label = ImageOperation.openImage(tmp_color_name)
    color_label.load()
    color_label = np.asarray(color_label)
    label = label_annotation(color_label)

    tmp_label_name = os.path.join(label_path, color_name)

    ImageOperation.saveImage(label, tmp_label_name)

def Color2label(color_path, label_path):
    if os.path.isdir(color_path):
        color_names = os.listdir(color_path)
    else:
        color_names = [color_path]

    Parallel(n_jobs=-1, backend="multiprocessing")(
        delayed(process_color)(color_path, label_path, color_name) for color_name in tqdm(color_names)
    )


def color_annotation(img):
    color = np.ones([img.shape[0], img.shape[1], 3])
    color[img == 0] = [255, 255, 255]  # white
    color[img == 1] = [0, 0, 0]  # black
    return Image.fromarray(np.uint8(color))

def Label2Color(label_path, color_label_path):
    if os.path.isdir(label_path):
        label_names = os.listdir(label_path)
    else:
        label_names = [label_path]

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []

        for i in range(len(os.listdir(label_path))):
            label_name = label_names[i]
            tmp_label_name = os.path.join(label_path, label_name)

            futures.append(executor.submit(process_image, tmp_label_name, color_label_path, label_name))

        for future in tqdm(as_completed(futures), total=len(futures)):
            pass

def process_image(tmp_label_name, color_label_path, label_name):
    label = ImageOperation.openImage(tmp_label_name)
    label.load()
    label = np.asarray(label)
    color_label = color_annotation(label)
    tmp_color_label_name = os.path.join(color_label_path, label_name)
    ImageOperation.saveImage(color_label, tmp_color_label_name)
if __name__ == '__main__':

    mask_dir = '/content/Land_Dataset/Wenchuan/mask/'       # Path to mask images
    label_dir = '/content/Land_Dataset/Wenchuan/label/'     # Path to save color labels

    # Ensure label directory exists
    os.makedirs(label_dir, exist_ok=True)

    Label2Color(mask_dir, label_dir)
    #Label2Color("./mask/", "./label/") # mask to label
    #Color2label("./label/",  "./mask/")  #label to mask



100%|██████████| 1/1 [00:00<00:00, 43.23it/s]


In [15]:
!pip install torch torchvision matplotlib tqdm


trying to implement U-net model mentioend in the paper

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=2):
        super(UNet, self).__init__()
        self.encoder1 = self.conv_block(in_channels, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.center = self.conv_block(512, 1024)
        self.decoder4 = self.conv_block(1024 + 512, 512)
        self.decoder3 = self.conv_block(512 + 256, 256)
        self.decoder2 = self.conv_block(256 + 128, 128)
        self.decoder1 = self.conv_block(128 + 64, 64)
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(F.max_pool2d(enc1, 2))
        enc3 = self.encoder3(F.max_pool2d(enc2, 2))
        enc4 = self.encoder4(F.max_pool2d(enc3, 2))
        center = self.center(F.max_pool2d(enc4, 2))
        dec4 = self.decoder4(torch.cat([F.interpolate(center, scale_factor=2, mode='bilinear', align_corners=True), enc4], 1))
        dec3 = self.decoder3(torch.cat([F.interpolate(dec4, scale_factor=2, mode='bilinear', align_corners=True), enc3], 1))
        dec2 = self.decoder2(torch.cat([F.interpolate(dec3, scale_factor=2, mode='bilinear', align_corners=True), enc2], 1))
        dec1 = self.decoder1(torch.cat([F.interpolate(dec2, scale_factor=2, mode='bilinear', align_corners=True), enc1], 1))
        return self.final_conv(dec1)


preparing the dataset for u-net

In [41]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class LandslideDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        # Map images to their corresponding masks by name
        self.images = sorted([img for img in os.listdir(image_dir) if img.lower().endswith('.tif')])
        self.masks = {mask.lower(): mask for mask in os.listdir(mask_dir) if mask.lower().endswith('.tif')}


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)

        # Find the corresponding mask by name (case insensitive)
        mask_name = self.masks.get(img_name.lower())
        if mask_name:
            mask_path = os.path.join(self.mask_dir, mask_name)
        else:
            raise FileNotFoundError(f"Mask file not found for image {img_name} at {self.mask_dir}")

        # Open image and mask
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # Apply transformations if defined
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


# Define transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# Load dataset
train_img_dir = '/content/Land_Dataset/Wenchuan/cropped_img/'
train_mask_dir = '/content/Land_Dataset/Wenchuan/label/'
train_dataset = LandslideDataset(train_img_dir, train_mask_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


train

In [42]:
import os
print(os.listdir('/content/Land_Dataset/Wenchuan/label/'))


['wenchuan098.TIF', 'wenchuan082.TIF', 'wenchuan045.TIF', 'wenchuan134.TIF', 'wenchuan146.TIF', 'wenchuan108.TIF', 'wenchuan112.TIF', 'wenchuan064.TIF', 'wenchuan083.TIF', 'wenchuan117.TIF', 'wenchuan075.TIF', 'wenchuan154.TIF', 'wenchuan109.TIF', 'wenchuan032.TIF', 'wenchuan035.TIF', 'wenchuan016.TIF', 'wenchuan051.TIF', 'wenchuan133.TIF', 'wenchuan156.TIF', 'wenchuan132.TIF', 'wenchuan103.TIF', 'wenchuan079.TIF', 'wenchuan140.TIF', 'wenchuan157.TIF', 'wenchuan057.TIF', 'wenchuan020.TIF', 'wenchuan028.TIF', 'wenchuan065.TIF', 'wenchuan084.TIF', 'wenchuan060.TIF', 'wenchuan004.TIF', 'wenchuan049.TIF', 'wenchuan115.TIF', 'wenchuan076.TIF', 'wenchuan040.TIF', 'wenchuan033.TIF', 'wenchuan120.TIF', 'wenchuan125.TIF', 'wenchuan080.TIF', 'wenchuan052.TIF', 'wenchuan068.TIF', 'wenchuan127.TIF', 'wenchuan023.TIF', 'wenchuan149.TIF', 'wenchuan046.TIF', 'wenchuan094.TIF', 'wenchuan042.TIF', 'wenchuan072.TIF', 'wenchuan014.TIF', 'wenchuan054.TIF', 'wenchuan141.TIF', 'wenchuan047.TIF', 'wenchuan01

In [46]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, masks in tqdm(train_loader):
        images, masks = images.to(device), masks.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

  0%|          | 0/23 [00:00<?, ?it/s]


FileNotFoundError: Mask file not found for image 0028.tif at /content/Land_Dataset/Wenchuan/label/

In [ ]:
from IPython.display import clear_output
import time

while True:
    print("Running to keep Colab alive...")
    time.sleep(60)
    clear_output(wait=True)

Running to keep Colab alive...
